In [1]:
using Pkg
Pkg.add("JuMP")
Pkg.add("GLPK")  # or any other suitable solver, like CPLEX, Gurobi, etc.

    Updating registry at `C:\Users\aidan\.julia\registries\General.toml`


   Resolving package versions...

   Installed Parsers ──────────── v2.8.0


   Installed OrderedCollections ─ v1.6.3


   Installed MathOptInterface ─── v1.22.0


...

    Updating `C:\Users\aidan\.julia\environments\v1.9\Project.toml`
  [60bf3e95] + GLPK v1.1.3
    Updating `C:\Users\aidan\.julia\environments\v1.9\Manifest.toml`


  [60bf3e95] + GLPK v1.1.3
  [e8aa6df9] + GLPK_jll v5.0.1+0
  [781609d7] + GMP_jll v6.2.1+2

Precompiling project...

  ✓ GLPK_jll


  ✓ GLPK
  2 dependencies successfully precompiled in 15 seconds. 39 already precompiled.

In [8]:
Pkg.add("Distributions")
using Distributions

   Resolving package versions...


  No Changes to `C:\Users\aidan\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\aidan\.julia\environments\v1.9\Manifest.toml`

2×3 Matrix{Float64}:
 12.2773  6.92142  8.36679
 12.4605  1.83434  5.8548

In [33]:
sedans = rand(Uniform(0,14), 2, 3)
hybrids = rand(Uniform(0,14), 2, 3)
mini = rand(Uniform(0,14), 2, 3)
vans = rand(Uniform(0,14), 2, 3)
trucks = rand(Uniform(0,14), 2, 4) #creating the age sets for each type of vehicle, dimensions are 2 by # of vehicles
fleet = [sedans, hybrids, mini, vans, trucks]
for i in 1:length(fleet)
    for k in 1:size(fleet[i],2)
        fleet[i][1,k] = round(fleet[i][1,k]) #round all vehicle ages to nearest year
        fleet[i][2,k] = 0 #boolean of whether a vehicle is electric
    end
end
fleetMiles = [4000,4000,4000,4000,12000] #based on US survey, car and van miles are 1/3 because fleet services says they are only checked out roughly 1/3 the time while trucks are mostly permanently assigned, same order as variables are initialized
gasCost = [0.18, 12, 23, 32, 24] #rough dollars of gas per mile driven, same order as above
elecCost = [0,0,0,0,0] #rough dollars of electricity per mile driven, same order
newGas = [0,0,0,0,0] #cost of buying a new gas vehicle of type n, same order
newElec = [0,0,0,0,0] #cost of buying a new electric vehicle of type n, same order


5-element Vector{Int64}:
 0
 0
 0
 0
 0

In [35]:
#yearly fleet operation cost
totalYearlyCost = 0
for i in 1:length(fleet)
    for k in 1:size(fleet[i],2)
        if fleet[i][2,k] == 0 #check for electric
            totalYearlyCost += fleetMiles[i]*gasCost[i]
        else 
            totalYearlyCost += fleetMiles[i]*elecCost[i] #could look to add a year modifyer to decrease electricity costs
        end
    end
end


In [ ]:
#function to replace vehicles, needs to be able to modify fleet array somehow

In [ ]:
using JuMP
using GLPK  # or your preferred solver

model = Model(GLPK.Optimizer)

# Decision variables
@variable(model, )

# Objective function (minimize the total cost)
@objective(model, Min, )

# Constraints (e.g., only one replacement per year)
